In [55]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import requests
import webbrowser
import urllib.request


def getReviews(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
    req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8')
    req.add_header('Accept-Language', 'en-US,en;q=0.5')
    response = urllib.request.urlopen(req)
    html_code = response.read().decode('utf-8')
    soup=BS(html_code,'html.parser')
    global totalReviews
    totalReviews=soup.find('span',class_='biGQs _P pZUbB KxBGd').text[:-8].replace(',','')
    hotelNameTemp=soup.find('h1',id='HEADING')
    if hotelNameTemp and hotelNameTemp.text.strip():
        hotelName=hotelNameTemp
    else:
        hotelName='N.A.'    
    reviewHeads=[i.text for i in soup.find_all('span',class_='JbGkU Cj')]
    reviewContent=[i.text for i in soup.find_all('span',class_='orRIx Ci _a C')]
    reviewerNdate = [i.text for i in soup.find_all('div', class_='tVWyV _Z o S4 H3 Ci') if 'wrote a review' in i.text]
    reviewerandDate=[]
    for i in reviewerNdate:
        reviewerandDate.append(i.split('wrote a review '))
    finalDate=[]
    finalreviewer=[]
    for i in reviewerandDate:
        finalDate.append(i[1])
        finalreviewer.append(i[0])
    ratings=[i.find('title').text[0:3] for i in soup.find_all('div',class_='kmMXA _T Gi')]
    tempHotelLocation= [i.text for i in soup.find_all('span',class_='biGQs _P pZUbB KxBGd')]
    hotelLocation=tempHotelLocation[1]
    df = pd.DataFrame({
        'Hotel Name': hotelName,
        'Hotel Location': hotelLocation,
        'Reviewer': finalreviewer,
        'Rating':ratings,
        'Date of Review': finalDate,
        'Review':reviewHeads,
        'Complete Review':reviewContent
    })
    return [df,hotelName,hotelLocation]

def getPureReviews(url,hotelName,hotelLocation):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
    req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8')
    req.add_header('Accept-Language', 'en-US,en;q=0.5')
    response = urllib.request.urlopen(req)
    html_code = response.read().decode('utf-8')
    soup=BS(html_code,'html.parser')
    reviewHeads=[i.text for i in soup.find_all('span',class_='JbGkU Cj')]
    reviewContent=[i.text for i in soup.find_all('span',class_='orRIx Ci _a C')]
    reviewerNdate = [i.text for i in soup.find_all('div', class_='tVWyV _Z o S4 H3 Ci') if 'wrote a review' in i.text]
    reviewerandDate=[]
    for i in reviewerNdate:
        reviewerandDate.append(i.split('wrote a review '))
    finalDate=[]
    finalreviewer=[]
    for i in reviewerandDate:
        finalDate.append(i[1])
        finalreviewer.append(i[0])
    ratings=[i.find('title').text[0:3] for i in soup.find_all('div',class_='kmMXA _T Gi')]
    df = pd.DataFrame({
        'Hotel Name': hotelName,
        'Hotel Location': hotelLocation,
        'Reviewer': finalreviewer,
        'Rating':ratings,
        'Date of Review': finalDate,
        'Review':reviewHeads,
        'Complete Review':reviewContent
    })
    return df

In [56]:
links_array = pd.read_excel(r"C:\Users\viren\Downloads\Country links.xlsx")['Unnamed: 0'].tolist()
for url in links_array[3:5]:
    split_url = url.split("Reviews-")
    first_part = split_url[0] + "Reviews-or"
    second_part = "-" + split_url[1]
    result = getReviews(url)
    df=result[0]
    hotelName=result[1]
    hotelLocation=result[2]
    urls = [f"{first_part}{i}{second_part}" for i in range(10, 30, 10)]
    for i in urls:
        newdf=getPureReviews(i,hotelName,hotelLocation)
        df = pd.concat([df, newdf], ignore_index=True)
    df.to_excel(f'{hotelName.text}.xlsx', index=False, engine='openpyxl')
    print(f'Converted {hotelName.text} to excel')

Converted Olá Santiago Providencia, Tapestry Collection By Hilton to excel
Converted Renaissance Santiago Hotel to excel


In [ ]:
url='https://www.tripadvisor.in/Hotel_Review-g294305-d7621139-Reviews-Courtyard_Santiago_Las_Condes-Santiago_Santiago_Metropolitan_Region.html'
df=getReviews(url)
df[0]